# Ideas

- load states.h5
- Search nearest kNN with annoy


In [56]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import annoy
import h5py

from annoy import AnnoyIndex

## How to build index etc (don't re-run!)

In [57]:
# Load the data
f = h5py.File("states.h5", "r")
cstar = f["cstar"]
print(cstar.shape)

(10000, 55, 500)


In [49]:
# Delete cstar to create a small copy
small_copy = h5py.File("states_small.h5", "w")
small_copy.create_group('src')
f.copy('src', small_copy['src'])
small_copy.create_group('tgt')
f.copy('tgt', small_copy['tgt'])
small_copy.create_group('attn')
f.copy('attn', small_copy['attn'])
small_copy.close()

In [58]:
# Create the index
t = AnnoyIndex(cstar.shape[2])

In [59]:
# Add samples to index, takes long!
for samplenum, sample in enumerate(cstar):
    for tokennum, tokencontext in enumerate(sample):
        index = cstar.shape[1] * samplenum + tokennum
        t.add_item(index, tokencontext)
#     if samplenum > 200:
#         break

In [60]:
# Build trees
t.build(10)

True

In [61]:
# Save index to file
t.save("states.ann")

True

## How to use the search

In [78]:
# Load the index from file
u = AnnoyIndex(500)
u.load("S2S/states.ann")

True

In [80]:
# Load the file with states etc
f = h5py.File("S2S/states_small.h5", "r")

In [73]:
# Define loader for dictionary and load them
def load_dict(fname):
    ix2w = {}
    with open(fname, "r") as f:
        for l in f:
            cline = l.split()
            ix2w[int(cline[0])] = cline[1]
    return ix2w

In [95]:
src_dict = load_dict("S2S/src.dict")
tgt_dict = load_dict("S2S/tgt.dict")

In [81]:
# Test functionality by giving index as input
def get_closest(ix, k=10):
    return u.get_nns_by_item(ix,k)
get_closest(1,5)

[210736, 534240, 232021, 306927, 112960]

In [85]:
# Index is stretched out, need to find src/tgt index
def convert_result_to_correct_index(oldix):
    return oldix // 55, oldix % 55
convert_result_to_correct_index(101)

(1, 46)

In [105]:
# Transform tokens, ignore padding (1)
def ix2text(array, vocab, highlight=-1):
    tokens = []
    for ix, t in enumerate(array):
        if ix == highlight:
            tokens.append("___" + vocab[t] + "___")
        elif t != 1:
            tokens.append(vocab[t])
    return " ".join(tokens)


In [131]:
# Compute length of a sentence when ignoring padding
def compute_sent_length(array):
    return np.sum([1 for t in array if t != 1])

In [155]:
# Convert a result for an index
def convert_result(ix):
    sentIx, tokIx = convert_result_to_correct_index(ix)
    # Get raw list of tokens
    src_in = f['src']['src'][sentIx]
    tgt_in = f['tgt']['tgt'][sentIx][1:]
    # Convert to text
    src = ix2text(src_in, src_dict)
    tgt = ix2text(tgt_in, tgt_dict, tokIx)
    # Extract attention - currently does not work as expected :()
    attn = f['attn']['attn'][sentIx]
    src_len = compute_sent_length(src_in)
    tgt_len = compute_sent_length(tgt_in)
    print(src)
    print(tgt)
    return src, tgt, attn
_ = convert_result(123)

It is not acceptable that , with the help of the national bureaucracies , Parliament &apos;s legislative prerogative should be made null and void by means of implementing provisions whose content , purpose and extent are not laid down in advance .
Es geht nicht an , dass über Ausführungsbestimmungen , deren Inhalt , Zweck ___und___ Ausmaß vorher nicht bestimmt ist , zusammen mit den nationalen Bürokratien das Gesetzgebungsrecht des Europäischen Parlaments ausgehebelt wird . </s>


In [ ]:
# Case study - we have 